# Prepare the data before the ML processes

***

## 1. Generate the clean dataframes 

In [ ]:
# DF with index sequences :

import pandas as pd 
import os 
path_labels = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_labeling/phageboost"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"
path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"

labels_results = ["protein_name","KL_type_ancestor","ancestor_id","IPR_entry","score","qstart","qstop","sequence"]
results_df = pd.read_csv(f"{path_db}/Results_III_DataFrame.v3.csv", sep="\t", names= labels_results)
sequence_df = pd.read_csv(f"{path_db}/Results_III_sequences.v3.csv", sep = "\t" ,names= ["index", "sequence"])

results_df["index_seq"] = sequence_df[sequence_df["sequence"]== results_df["sequence"]]["index"]

index_list = []
for index , row in tqdm(results_df.iterrows()) :
    index_seq = sequence_df[sequence_df["sequence"] == row["sequence"]]["index"].values[0]
    index_list.append(index_seq)
    
results_df["index_seq"] = index_list
results_df.to_csv(f"{path_db}/Results_III_DataFrame.v3.final.csv", sep="\t",header =  ["protein_name","KL_type_ancestor","ancestor_id","IPR_entry","score","qstart","qstop","sequence","index_seq"], index = False )


> The chosen ones : removing the undesired IPR hits 

In [ ]:
import pandas as pd 
from tqdm import tqdm

results_df = pd.read_csv(f"{path_db}/Results_III_DataFrame.v3.final.csv", sep="\t", header= 0)

indices = set(list(results_df.index_seq)) # correct number 

tail = ["IPR005604"]
undesirable = ["IPR023346", "IPR013189","IPR002152","IPR013830","IPR013189","IPR000490","IPR000852"]




def filter_ipr(ipr) :
    undesired_df = results_df[results_df["IPR_entry"].str.split(".").str[0] == ipr]
    unique_df = undesired_df.drop_duplicates(subset="index_seq", keep="first")
    return unique_df

not_processed = set()
for ipr in undesirable :
    undesired_df = filter_ipr(ipr)
    not_processed.update(list(undesired_df["index_seq"]))
    


tail_df = results_df[results_df["IPR_entry"].str.split(".").str[0] == "IPR005604"]
tail_df = filter_ipr("IPR015165")
poly_df = filter_ipr("IPR024535")
pectate_df = filter_ipr("IPR024535")
endo_sia_df = filter_ipr("IPR036278")

***
## 2. Basic Information

#### **A. About the most virulent KL types**

In [ ]:
import os
import random
import subprocess
from tqdm import tqdm
from random import sample
import time
import pandas as pd
from collections import Counter


path_phageboost_pred="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_prediction"

path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_database_PFAM = "/home/conchae/databases/Pfam-A/pfam"
path_depolymerse_w = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/20102022_session"
path_label = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_labeling/phageboost/info"

label_names = ["Prophage_name","K_serotype_monophyletic_group", "Ancestor","Number_ancestors","Size_clade","Min_ancestors","Number_Ktypes","K_types","Number_Kswaps"]

depolymerase_F_df = pd.read_csv(f"{path_depolymerse_w}/depolymerase_proteins.PFAM_depo.final.tsv", sep="\t" , header = 0)
LLHP_F_df = pd.read_csv(f"{path_depolymerse_w}/LLHP_proteins.PFAM_depo.clean.txt", sep="\t",names = df_names)

dic_counter = dict(Counter(depolymerase_F_df["Label"]))
info_depolymerase = {}

for K_type, count in dic_counter.items() :
    if count > 10 :
        info_depolymerase.update({K_type : count})
        
info_depolymerase = {'KL47': 139, 'KL2': 135, 'KL107': 135, 'KL22': 35, 'KL102': 49, 'KL51': 30, 'KL126': 12, 'KL17': 98, 'KL52': 13, 'KL24': 33, 'KL27': 67,
                     'KL39': 13, 'KL3': 55, 'KL64': 169, 'KL25': 34, 'KL60': 23, 'KL30': 48, 'KL46': 28, 'KL137': 11, 'KL106': 37, 'KL21': 25, 'KL45': 35, 'KL13': 18,
                     'KL169': 13, 'KL14': 39, 'KL105': 23, 'KL8': 16, 'KL10': 26, 'KL81': 14, 'KL1': 39, 'KL151': 41, 'KL16': 11, 'KL136': 11, 'KL74': 14, 'KL7': 30,
                     'KL62': 28, 'KL43': 18, 'KL145': 11, 'KL57': 23, 'KL53': 12, 'KL23': 18, 'KL61': 16, 'KL127': 14, 'KL18': 13}
